# Tensorflow 1 - Tensor 及相关数据结构

In [3]:
import tensorflow as tf

## 1 tensor 基本操作

### 1-1 tf.constant() 创建向量

In [2]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

<tf.Tensor: id=1, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42) # scalar

<tf.Tensor: id=2, shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape

TensorShape([2, 3])

In [6]:
t.dtype

tf.float32

### 1-2 Indexing 索引

In [7]:
t[:, 1:]

<tf.Tensor: id=7, shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis]

<tf.Tensor: id=11, shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

### 1-3 Ops 操作

* 有些操作名称相同，如 tf.add(), tf.multiply(), tf.square(), tf.exp(), tf.sqrt(), tf.reshape(), tf.squeeze(), tf.tile().
* 有些操作名称不同，如 tf.reduce_mean() -- np.mean(), tf.reduce_sum() -- np.sum(), tf.reduce_max() -- np.max(), tf.math.log() -- np.log().

In [9]:
t + 10

<tf.Tensor: id=13, shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [10]:
tf.square(t)

<tf.Tensor: id=14, shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [11]:
t @ tf.transpose(t) # tf.matmul(), 没有 t.T

<tf.Tensor: id=17, shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

### 1-4 keras.backend

* 若要编写可移植到其他 keras 中的实现代码，应该使用 keras.backend。

In [12]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: id=22, shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### 1-5 Tensor <-> Numpy

* tensor 通常使用 32 位精度。而 numpy 默认 64 位精度。所以 numpy 数组建立张量应确保 dtype=tf.float32

In [19]:
import numpy as np

a = np.array([2., 4., 5.], dtype=np.float32)
tf.constant(a)

<tf.Tensor: id=26, shape=(3,), dtype=float32, numpy=array([2., 4., 5.], dtype=float32)>

In [20]:
t.numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: id=3, shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>>

In [21]:
tf.square(a)

<tf.Tensor: id=28, shape=(3,), dtype=float32, numpy=array([ 4., 16., 25.], dtype=float32)>

In [22]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### 1-6 Conflicting types

* Tensor 不同类型之间不能自动转换。
* tf.cast() 函数可以实现转换。

In [23]:
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: add/


In [24]:
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: add/


In [25]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: id=36, shape=(), dtype=float32, numpy=42.0>

## 2 相关数据结构

### 2-1 Strings

In [29]:
tf.constant(b'Hello world') # 字节字符串

<tf.Tensor: id=40, shape=(), dtype=string, numpy=b'Hello world'>

In [32]:
tf.constant("café") # utf-8 编码

<tf.Tensor: id=43, shape=(), dtype=string, numpy=b'caf\xc3\xa9'>

In [34]:
u = tf.constant([ord(c) for c in 'café']) # tf.int32 张量表示
u

<tf.Tensor: id=45, shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233], dtype=int32)>

In [35]:
b = tf.strings.unicode_encode(u, 'UTF-8') # int32 转 utf8
tf.strings.length(b, unit='UTF8_CHAR') # 4

<tf.Tensor: id=56, shape=(), dtype=int32, numpy=4>

In [37]:
tf.strings.unicode_decode(b, 'UTF-8') # utf8 转 int32

<tf.Tensor: id=60, shape=(4,), dtype=int32, numpy=array([ 99,  97, 102, 233], dtype=int32)>

### 2-2 String Arrays

In [38]:
p = tf.constant(["Café", "Coffee", "caffè", "咖啡"])

In [39]:
tf.strings.length(p, unit='UTF8_CHAR') # 返回每个元素的长度

<tf.Tensor: id=62, shape=(4,), dtype=int32, numpy=array([4, 6, 5, 2], dtype=int32)>

In [40]:
r = tf.strings.unicode_decode(p, 'UTF8') # 每个元素的 UTF8 编码
r # 不规则张量

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [21654, 21857]]>

### 2-3 Ragged Tensors 不规则张量

In [42]:
print(r[1])

tf.Tensor([ 67 111 102 102 101 101], shape=(6,), dtype=int32)


In [43]:
print(r[1:3])

<tf.RaggedTensor [[67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232]]>


In [44]:
r2 = tf.ragged.constant([[65, 66], [], [67]])
print(tf.concat([r, r2], axis=0))

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [21654, 21857], [65, 66], [], [67]]>


In [46]:
r3 = tf.ragged.constant([[68, 69, 70], [71], [], [72, 73]])
print(tf.concat([r, r3], axis=1))

<tf.RaggedTensor [[67, 97, 102, 233, 68, 69, 70], [67, 111, 102, 102, 101, 101, 71], [99, 97, 102, 102, 232], [21654, 21857, 72, 73]]>


In [47]:
tf.strings.unicode_encode(r3, 'UTF-8')

<tf.Tensor: id=214, shape=(4,), dtype=string, numpy=array([b'DEF', b'G', b'', b'HI'], dtype=object)>

In [48]:
r.to_tensor()

<tf.Tensor: id=279, shape=(4, 6), dtype=int32, numpy=
array([[   67,    97,   102,   233,     0,     0],
       [   67,   111,   102,   102,   101,   101],
       [   99,    97,   102,   102,   232,     0],
       [21654, 21857,     0,     0,     0,     0]], dtype=int32)>

### 2-4 Sparse Tensors 稀疏向量

* 只要给一些 indices 和 values 就能制造一个稀疏矩阵：

In [49]:
s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3]], # 有值的位置
                    values=[1., 2., 3.], # 数据
                    dense_shape=[3, 4]) # Tensor形状
print(s)

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


* to_dense 还原：

In [50]:
tf.sparse.to_dense(s)

<tf.Tensor: id=284, shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

* 稀疏矩阵不能广播：

In [52]:
try:
    s3 = s + 1
except TypeError as ex:
    print(ex)

unsupported operand type(s) for +: 'SparseTensor' and 'int'


* 可以和一般矩阵相乘：

In [53]:
s4 = tf.constant([[10., 20.], [30., 40.], [50., 60.], [70., 80.]])
tf.sparse.sparse_dense_matmul(s, s4) # 与稀疏矩阵的乘法

<tf.Tensor: id=286, shape=(3, 2), dtype=float32, numpy=
array([[ 30.,  40.],
       [ 20.,  40.],
       [210., 240.]], dtype=float32)>

* 稀疏矩阵在操作时需要顺序给 indices，否则会报错：

In [55]:
s5 = tf.SparseTensor(indices=[[0, 2], [0, 1]],
                     values=[1., 2.],
                     dense_shape=[3, 4])
print(s5)

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]], shape=(2, 2), dtype=int64), values=tf.Tensor([1. 2.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


In [56]:
try:
    tf.sparse.to_dense(s5)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

indices[1] = [0,1] is out of order [Op:SparseToDense]


* 解决方法是 reorder 一下：

In [57]:
s6 = tf.sparse.reorder(s5)
tf.sparse.to_dense(s6)

<tf.Tensor: id=295, shape=(3, 4), dtype=float32, numpy=
array([[0., 2., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### 2-5 Sets

* 集合用常规向量表示。可以用 tf.sets 包来操作
* 最后多出来的 0 不是结果。

In [58]:
set1 = tf.constant([[2, 3, 5, 7], [7, 9, 0, 0]])
set2 = tf.constant([[4, 5, 6], [9, 10, 0]])
tf.sparse.to_dense(tf.sets.union(set1, set2)) # 纵向求并

<tf.Tensor: id=302, shape=(2, 6), dtype=int32, numpy=
array([[ 2,  3,  4,  5,  6,  7],
       [ 0,  7,  9, 10,  0,  0]], dtype=int32)>

In [59]:
tf.sparse.to_dense(tf.sets.difference(set1, set2)) # 求差集

<tf.Tensor: id=307, shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 7],
       [7, 0, 0]], dtype=int32)>

In [60]:
tf.sparse.to_dense(tf.sets.intersection(set1, set2)) # 求交

<tf.Tensor: id=312, shape=(2, 2), dtype=int32, numpy=
array([[5, 0],
       [0, 9]], dtype=int32)>

### 2-6 Variables

* tf.Tensor 值不变，因此无法用于在神经网络中实现权重。所以产生了 tf.Variable.
* 通过 .assign() 方法进行赋值。不能直接用 = 赋值。

In [61]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])

In [62]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [63]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [64]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [65]:
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment


* 分散式更新：用.scatter_nd_update() 进行更新。

In [66]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

In [67]:
sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[1, 0])
v.scatter_update(sparse_delta)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>

### 2-7 Tensor Arrays 张量列表

In [72]:
array = tf.TensorArray(dtype=tf.float32, size=3)
array = array.write(0, tf.constant([1., 2.]))
array = array.write(1, tf.constant([3., 10.]))
array = array.write(2, tf.constant([5., 7.]))

In [74]:
array.read(1) # 读完就没了

<tf.Tensor: id=355, shape=(2,), dtype=float32, numpy=array([ 3., 10.], dtype=float32)>

In [75]:
array.stack()

<tf.Tensor: id=361, shape=(3, 2), dtype=float32, numpy=
array([[1., 2.],
       [0., 0.],
       [5., 7.]], dtype=float32)>

In [76]:
mean, variance = tf.nn.moments(array.stack(), axes=0)
mean

<tf.Tensor: id=369, shape=(2,), dtype=float32, numpy=array([2., 3.], dtype=float32)>

In [77]:
variance

<tf.Tensor: id=370, shape=(2,), dtype=float32, numpy=array([4.6666665, 8.666667 ], dtype=float32)>